In [1]:
import os
import sys 
from flask import Flask
from flask import request
from flask_api import status
import psycopg2
import json
import copy
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
datosBaseDeDatos = {"usuario":"postgres","password":"123456789","Host":"127.0.0.1","puerto":"5432","DB":"chatblocks"}

In [2]:
def abrirJson(name,path=os.getcwd()+"/" ):
    with open(path+name,encoding='utf-8') as data_file:  
        diccionario = json.load(data_file)
    return diccionario

In [3]:
def conexionBaseDeDatos(datosConexion):
    try:
        connection = psycopg2.connect(user = datosConexion["usuario"],
                              password = datosConexion["password"],
                              host = datosConexion["Host"],
                              port = datosConexion["puerto"],
                              database = datosConexion["DB"])    
        return connection
    except(Exception, psycopg2.Error) as error:
        print("Error al conectar con base de datos", error)

In [4]:
def consultar(connection,datos=False,consulta=False,tabla=False):
    #print("datos",datos)
    consultaBots = list()
    column_names = []
    data_rows = []
    diccionarioDatos = dict()
    consultas = list()
    cursor = connection.cursor()  
    cosulta =""    
    if datos:
        #print("intenatando consulta...")
        consulta = "select * from "+tabla+" where "        
        i=0
        for dato, valorDato in datos.items():
            if i==0:
                consulta = consulta+dato+"= '"+str(valorDato)+"'"                
            else:             
                consulta = consulta+" and "+dato+"= '"+str(valorDato)+"'" 
            i=i+1
        #print(dato,":",valorDato)
        consulta= consulta + " ;" 
        #print(consulta)        
        #consulta = "select * from robots where name_robot = '"+datosBot["name_robot"]+"' and id_face = '"+ datosBot["id_face"]+"' and  type_blocki = '"+datosBot["type_blocki"]+"' and access_token= '"+datosBot["access_token"]+"' and id_user = '"+datosBot["id_user"]+"';"
    print("consulta:",consulta)
    cursor.execute(consulta)    
    column_names = [desc[0] for desc in cursor.description]  
    rows = cursor.fetchall()
    for row in rows:
        diccionarioDatos = dict()
        for i in range(len(column_names)):
            diccionarioDatos.update({column_names[i]:row[i]})
        consultaBots.append(diccionarioDatos)
    cursor.close()
    connection.close()  
    
    if len(consultaBots)==1:
        return consultaBots[0]
    else:
        return consultaBots

In [5]:
def insert(datos,tabla,connection):
    cursor = connection.cursor()
    values = ""
    datosInsert = ""
    i = 0;
    for dato, valorDato in datos.items():
        if i==0:
            values = values+dato 
            datosInsert = datosInsert+"'"+str(valorDato)+"' "                
        else:
            values = values+","+dato
            datosInsert =datosInsert+",'"+str(valorDato)+"' "
        i=i+1
    insersion  = "insert into "+tabla+" ( "+values+" ) values ("+datosInsert+" );" 
    cursor.execute(insersion)
    connection.commit()
    cursor.close()
    connection.close()
    return datos

In [6]:
def update(datosActualizar,datos,tabla,connection):
    actualizar =" "
    actualizar = "update "+tabla+" set"
    j = 0
    for dato,informacionDato in datosActualizar.items():
        print(dato)
        if j==0:
            actualizar=actualizar+" "+dato+"="+str(informacionDato)
            j=j+1
        else:
             actualizar=actualizar+", "+dato+"="+str(informacionDato)
    
    i=0
    actualizar=actualizar+" where "
    for dato, valorDato in datos.items():
        if i==0:
            actualizar = actualizar+dato+"= '"+str(valorDato)+"'"                
        else:             
            actualizar = actualizar+" and "+dato+"= '"+str(valorDato)+"'" 
        i=i+1
    actualizar= actualizar + " ;"  

In [7]:
app = Flask(__name__)
@app.route('/access_token', methods = ['POST'])
def postAccessToken():
    try:         
        connection = conexionBaseDeDatos(datosBaseDeDatos)        
        datosVariable = request.get_json()   
        tabla = "robots"
        con = consultaBots(connection,datos=datosVariable,tabla=tabla)        
        listaAcessToken=list()
        if type(con)==list:
            for i in range(len(con)):
                print(con[i])
                listaAcessToken.append(con[i]["access_token"])
        else:
            listaAcessToken.append(con["access_token"])         
        print(listaAcessToken)
        return {"access_token":listaAcessToken}
    except:
        return datosVariable,status.HTTP_404_NOT_FOUND


In [8]:
@app.route('/insert/<tabla>',methods=['POST'])
def funcionalidad(tabla):
    try:
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datos = request.get_json()
        print(datos)
        insert(datos,str(tabla),connection)
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        consulta = consultar(connection,datos=datos,tabla=str(tabla))
        return consulta
    except (Exception, psycopg2.Error) as error :
        print("error",error)
        return tabla,status.HTTP_404_NOT_FOUND

In [9]:
if __name__ == "__main__":    
    app.run(host='localhost',port='8181')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8181/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/robots HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/variables HTTP/1.1" 200 -


{'name_robot': 'oliverBot', 'id_face': '430761350747124', 'block_ini': 'mensajesIntroduccion_01inicial_inicial', 'type_blocki': 'case', 'access_token': 'EAAJjZAQWaAbgBAAxC53jaIzoxSpgY3qZBc0pYDlb5g3E58pTJ35vAzrsZBVSyiYPhpgZB3ZAeNqx18ImA0LZCcsqL96hFDqsE9fcXUk52ofVsqWeerVDKp0cGo2wvohq5ISdkCUX7ZByd4c80Y6BstQYbe5noQhC8L7wdtMuKyZAAgZDZD', 'id_user': 'oliverbp88@gmail.com', 'api_nlp': 'Sin Api'}
consulta: select * from robots where name_robot= 'oliverBot' and id_face= '430761350747124' and block_ini= 'mensajesIntroduccion_01inicial_inicial' and type_blocki= 'case' and access_token= 'EAAJjZAQWaAbgBAAxC53jaIzoxSpgY3qZBc0pYDlb5g3E58pTJ35vAzrsZBVSyiYPhpgZB3ZAeNqx18ImA0LZCcsqL96hFDqsE9fcXUk52ofVsqWeerVDKp0cGo2wvohq5ISdkCUX7ZByd4c80Y6BstQYbe5noQhC8L7wdtMuKyZAAgZDZD' and id_user= 'oliverbp88@gmail.com' and api_nlp= 'Sin Api' ;
{'name_var': 'contador_mensajesIntroduccion', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'Numerico', 'id_robot': 338}
consulta: select * from variables where name_var= 'con

127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/bloquecase HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/bloqueinternalprocess HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/variables HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/bloquequickreply HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/variables HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/bloquecase HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:13:15] "POST /insert/variables HTTP/1.1" 200 -


consulta: select * from bloquecase where default_id= 'menuIntroduccion_01inicial_inicial' and namestate= 'mensajesIntroduccion_01inicial_inicial' and method= 'GET' and typing_messages= '3' and url= 'http://127.0.0.1:1111/increase/contador_mensajesIntroduccion/3' and messages= '¡Hola! Soy Niza.,Un asistente virtual para atención ciudadana puedo responder dudas sobre tu agua potable,Te gustaría saber cómo utilizarme' and default_message= 'No te entiendo' and type_default_message= 'text' and type_messages= 'text,text,text' and options= 'mensajesIntroduccion_01inicial_inicial,mensajesIntroduccion_01inicial_inicial,mensajesIntroduccion_01inicial_restarVariable' and id_robot= '338' and id_var= '816' and blocktype= 'case' ;
{'next_true': 'menuIntroduccion_01inicial_inicial', 'next_false': 'menuIntroduccion_01inicial_inicial', 'namestate': 'mensajesIntroduccion_01inicial_restarVariable', 'method': 'GET', 'url': 'http://127.0.0.1:1111/restar/contador_mensajesIntroduccion', 'id_robot': 338, 'blo

...

consulta: select * from bloquecase where default_id= 'menuIntroduccion_01inicial_inicial' and namestate= 'mensajesIntroduccion_01inicial_inicial' and method= 'GET' and typing_messages= '3' and url= 'http://127.0.0.1:1111/increase/contador_mensajesIntroduccion/3' and messages= '¡Hola! Soy Niza.,Un asistente virtual para atención ciudadana puedo responder dudas sobre tu agua potable,Te gustaría saber cómo utilizarme' and default_message= 'No te entiendo' and type_default_message= 'text' and type_messages= 'text,text,text' and options= 'mensajesIntroduccion_01inicial_inicial,mensajesIntroduccion_01inicial_inicial,mensajesIntroduccion_01inicial_restarVariable' and id_robot= '340' and id_var= '828' and blocktype= 'case' ;
{'next_true': 'menuIntroduccion_01inicial_inicial', 'next_false': 'menuIntroduccion_01inicial_inicial', 'namestate': 'mensajesIntroduccion_01inicial_restarVariable', 'method': 'GET', 'url': 'http://127.0.0.1:1111/restar/contador_mensajesIntroduccion', 'id_robot': 340, 'blo

127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloquecase HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/variables HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloquecase HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloqueinternalprocess HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/variables HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloquequickreply HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloqueinformativo HTTP/1.1" 200 -


consulta: select * from bloquecase where default_id= 'qrPersonalidad_01inicial_inicial' and namestate= 'mensajePreMenu_01inicial_inicial' and method= 'GET' and typing_messages= '3' and url= 'http://127.0.0.1:1111/random/qrd_mensaje_premenu/3' and messages= '¿Ahora en qué puedo ayudarte?,¿Cómo puedo ayudarte? ,¿Qué te gustaría hacer?' and default_message= 'no te entiendo' and type_default_message= 'text' and type_messages= 'text,text,text' and options= 'qrPersonalidad_01inicial_inicial,qrPersonalidad_01inicial_inicial,qrPersonalidad_01inicial_inicial' and id_robot= '340' and id_var= '830' and blocktype= 'case' ;
{'name_var': 'mensage_intro2', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'Numerico', 'id_robot': 340}
consulta: select * from variables where name_var= 'mensage_intro2' and opc_type= 'Variable' and var= '0' and opc_data= 'Numerico' and id_robot= '340' ;
{'default_id': 'mensajesIntroduccion_01inicial_inicial', 'namestate': 'mensajesIntroduccion2_01inicial_inicial', 'method': 

127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloqueinformativo HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/variables HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloqueinput HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 12:32:38] "POST /insert/bloquenlp HTTP/1.1" 200 -


{'name_var': 'var_pregunta', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'String', 'id_robot': 340}
consulta: select * from variables where name_var= 'var_pregunta' and opc_type= 'Variable' and var= '0' and opc_data= 'String' and id_robot= '340' ;
{'next_id': 'NLPpersonalidad_01inicial_inicial', 'default_id': 'NLPpersonalidad_01inicial_inicial', 'namestate': 'pregunta_01inicial_inicial', 'contenido': '¿Qué deseas hacer?', 'contenttype': 'text', 'typingtime': 1, 'validacion': '.*', 'id_robot': 340, 'id_var': 833, 'blocktype': 'input'}
consulta: select * from bloqueinput where next_id= 'NLPpersonalidad_01inicial_inicial' and default_id= 'NLPpersonalidad_01inicial_inicial' and namestate= 'pregunta_01inicial_inicial' and contenido= '¿Qué deseas hacer?' and contenttype= 'text' and typingtime= '1' and validacion= '.*' and id_robot= '340' and id_var= '833' and blocktype= 'input' ;
{'default_id': 'mensajesIntroduccion_01inicial_inicial', 'namestate': 'NLPpersonalidad_01inicial_inicial', 'def